In [1]:
import logging
import os,re,tqdm
import datetime

import pandas as pd
import numpy as np
import joblib,pathlib

from transformers import BertTokenizer, BertModel
import torch

from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:

DATA_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"data/translated_data") #take the data translated 
DATA_PATH_CSV = os.path.join(os.path.dirname(os.getcwd()), "data/chunks")

try:
    data_text = pd.read_csv(os.path.join(DATA_PATH,os.listdir(DATA_PATH)[0]),index_col=0,encoding='utf-8')
    for path_chunk in os.listdir(DATA_PATH)[1:]:
        data_text = pd.concat( [data_text,pd.read_csv(os.path.join(DATA_PATH,path_chunk),index_col=0,encoding='utf-8')],ignore_index=True)
    print('data loaded')
except Exception as e:
    print(f"Failed to load and concatenate data: {e}")

data loaded


157755

In [4]:
# data loading
DATA_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data/chunks")
try:
    data_text = pd.read_csv(os.path.join(DATA_PATH, os.listdir(DATA_PATH)[0]), index_col=0, encoding='utf-8')
    logging.info("Data loaded")
except Exception as e:
    logging.error(f"Failed to load and concatenate data: {e}")
    raise

#get all questions in a list
questions = data_text.question.values.tolist()
# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
# Load the BERT model.
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [3]:
#Main
if __name__ == "__main__":

   

    DATA_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"data/chunks") # pathlib.Path("/workspaces/Assist_tickets/data/chunks")

    data_text = pd.read_csv(os.path.join(DATA_PATH,os.listdir(DATA_PATH)[0]),index_col=0,encoding='utf-8')
    print('data loaded')
    
    #get all questions in a list
    questions = data_text.question.values.tolist() 

    # Load the BERT tokenizer.
    print('Loading MBERT tokenizer.')
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False) # all my question are lower, that's why I put do_lower_case=False
    print('Loading MBERT model.')
    model = BertModel.from_pretrained('bert-base-multilingual-cased')

    # question is a list of sentence
    batch_size = 32  # How can find the max size of the batch size ??

    def batchify(data, batch_size):
        """return list of list len batch size [["...","...","..."],["...","...","..."]] """
        return [data[i:i + batch_size] for i in range(0, len(data), batch_size)] 

    
    question_batches = batchify(questions[:64], batch_size) #[["...","...","..."],["...","...","..."],["...","...","..."],["...","...","..."]]
    sentence_embeddings = []

    for batch in question_batches:# add a description : desc="Processing batches" 
        # here a batch = ["...","...","..."]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad(): #don't calculate the gradiant to optimize the RAM
            outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            batch_embeddings = torch.mean(last_hidden_states, dim=1) # take the barycenter (mean) of all embedding
            sentence_embeddings.append(batch_embeddings)

        # Concat all embedding
        sentence_embeddings = torch.cat(sentence_embeddings, dim=0)


data loaded
Loading MBERT tokenizer.
Loading MBERT model.


AttributeError: 'Tensor' object has no attribute 'append'